In [ ]:
import astropy.units as u
import lsdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dask

from dask.distributed import Client
from nested_pandas.utils import count_nested
from dask.distributed import print as dask_print
from lsdb.core.search.pixel_search import PixelSearch
from lsdb.core.search.order_search import OrderSearch
from io import StringIO
from nested_pandas import NestedDtype
from pathlib import Path

In [ ]:
# in_memory.rel_dev.values.flatten().ravel()
np.concatenate(in_memory.rel_dev.values).ravel().shape

In [ ]:
x = np.arange(-10, 10, 0.1)
import scipy

_ = plt.hist(np.concatenate(in_memory.rel_dev.values).ravel(), bins=x, density=True)
plt.plot(x, scipy.stats.norm().pdf(x))
# scipy.stats.norm().pdf(x)